In [1]:
import sys
import os

from dotenv import load_dotenv

sys.path.insert(0, os.path.abspath("../src"))
print(sys.path)

load_dotenv()

['/Users/I551385/Code/Personal/LLMsAgents-TextToSQL/src', '/opt/homebrew/Cellar/python@3.12/3.12.6/Frameworks/Python.framework/Versions/3.12/lib/python312.zip', '/opt/homebrew/Cellar/python@3.12/3.12.6/Frameworks/Python.framework/Versions/3.12/lib/python3.12', '/opt/homebrew/Cellar/python@3.12/3.12.6/Frameworks/Python.framework/Versions/3.12/lib/python3.12/lib-dynload', '', '/Users/I551385/Library/Caches/pypoetry/virtualenvs/llmsagents-texttosql-bQPeKkIa-py3.12/lib/python3.12/site-packages']


True

In [2]:
import pandas as pd

df = pd.read_json("../data/dev/dev.json")

df.head()

,question_id,db_id,question,evidence,SQL,difficulty
0,0,california_schools,What is the highest eligible free rate for K-1...,Eligible free rate for K-12 = `Free Meal Count...,SELECT `Free Meal Count (K-12)` / `Enrollment ...,simple
1,1,california_schools,Please list the lowest three eligible free rat...,Eligible free rates for students aged 5-17 = `...,SELECT `Free Meal Count (Ages 5-17)` / `Enroll...,moderate
2,2,california_schools,Please list the zip code of all the charter sc...,Charter schools refers to `Charter School (Y/N...,SELECT T2.Zip FROM frpm AS T1 INNER JOIN schoo...,simple
3,3,california_schools,What is the unabbreviated mailing street addre...,,SELECT T2.MailStreet FROM frpm AS T1 INNER JOI...,simple
4,4,california_schools,Please list the phone numbers of the direct ch...,Charter schools refers to `Charter School (Y/N...,SELECT T2.Phone FROM frpm AS T1 INNER JOIN sch...,moderate


In [3]:
percentages = df["difficulty"].value_counts(normalize=True) * 100

print(percentages)

difficulty
simple         60.299870
moderate       30.247718
challenging     9.452412
Name: proportion, dtype: float64


In [4]:
sample = df.sample(100 , random_state=0)

sample_distribution = sample["difficulty"].value_counts(normalize=True) * 100

sample_distribution

difficulty
simple         63.0
moderate       30.0
challenging     7.0
Name: proportion, dtype: float64

In [5]:
from approach_evaluator import avg_execution_time
import numpy as np

sample["execution_time"] = avg_execution_time(sample["db_id"], sample["SQL"])

sample.head()

KeyboardInterrupt: 

In [ ]:
os.makedirs("../sample", exist_ok=True)

sample.to_csv("../sample/dev2.csv", index=False)

### Add noisy queries

In [19]:
import random
import re


sample_for_noise = df.sample(500 , random_state=0)[100:]

# List of random words to replace string literals in SQL queries
random_words = ["apple", "banana", "cherry", "dragonfly", "elephant", "falcon", "giraffe", "horizon", "igloo"]

def replace_with_random_word():
    """Replace a word with a random one from the list."""
    return random.choice(random_words)

def replace_string_literals(query, replacement_probability=0.7):
    """Replace a random subset of string literals in SQL query with random words."""
    # Regular expression to match string literals (anything between single quotes)
    def replace_string_literal(match):
        # Decide randomly whether to replace the string literal
        if random.random() < replacement_probability:
            return f"'{replace_with_random_word()}'"
        return match.group(0)

    # Apply the replacement for each string literal in the query
    return re.sub(r"'([^']*)'", replace_string_literal, query)

# Define random error generators
def introduce_logical_error(query):
    """Introduce a logical error by flipping ASC/DESC."""
    if "DESC" in query:
        return query.replace("DESC", "ASC")
    if "ASC" in query:
        return query.replace("ASC", "DESC")
    return query

def introduce_semantic_error(query):
    """Introduce a semantic error by mismatching aggregate functions."""
    if "MAX(" in query:
        return query.replace("MAX(", "MIN(")
    if "SUM(" in query:
        return query.replace("SUM(", "AVG(")
    return query


def replace_integers_with_random(query):
    """Replace integers with random integers, avoiding table alias references."""
    
    # Regular expression to match integers, but avoid replacing those in table.column notation (e.g., T1.xyz)
    def replace_integer(match):
        # If the match is a table alias with a dot (e.g., T1.xyz), don't replace it
        if re.match(r'^[A-Za-z0-9_]+(?:\.[A-Za-z0-9_]+)$', match.group(0)):
            return match.group(0)
        # Otherwise, replace the integer with a random number
        return str(random.randint(1, 1000))

    return re.sub(r'\b\d+\b(?!\.)', replace_integer, query)

# Function to randomly corrupt SQL queries
def corrupt_query(row):
    query = row["SQL"]
    error_generators = [
        introduce_logical_error,
        replace_string_literals,
        introduce_semantic_error,
        replace_integers_with_random
    ]
    
    # Randomly choose a subset of errors to apply
    num_errors = random.randint(1, len(error_generators))
    chosen_errors = random.sample(error_generators, num_errors)
    
    # Apply chosen errors
    for error_func in chosen_errors:
        query = error_func(query)
    return query


sample_for_noise["corrupted"] = df.apply(corrupt_query, axis=1)

sample_for_noise["noised"] = sample_for_noise["SQL"] != sample_for_noise["corrupted"]

sample_for_noise["SQL"] = sample_for_noise["corrupted"]
sample_for_noise.drop(columns=["corrupted"], inplace=True)

sample_for_noise = sample_for_noise[sample_for_noise["noised"]]

In [20]:
noised_count = sample_for_noise["noised"].sum()
print(f"Number of noised queries: {noised_count}")

Number of noised queries: 288


In [21]:
from tools.database import Database

def execute_query(query, database):
    try:
        db = Database(database)
        results, _ = db.execute_query(query)
        return results
    except Exception as e:
        return False


executed_count = 0
executed_empty_count = 0
not_executed_count = 0

# List to store indexes of executed queries
executed_indexes = []

for s in sample_for_noise.itertuples():
    if s.noised:
        result = execute_query(s.SQL, s.db_id)
        if result is not False:
            if len(result) == 0:
                executed_empty_count += 1
                print(f"Query executed empty at index {s.Index}")

            else:
                executed_count += 1
                print(f"Query executed at index {s.Index}") # Store the index of executed query
                executed_indexes.append(s.Index) 
        else:
            not_executed_count += 1
            print(f"Query is not executed at index {s.Index}")

# After processing, print the results
print(f"Total executed queries: {executed_count}")
print(f"Total executed empty queries: {executed_empty_count}")
print(f"Total not executed queries: {not_executed_count}")

# Optionally, print the list of indexes of executed queries
print(f"Indexes of executed queries: {executed_indexes}")


Query executed at index 1195
Query executed at index 1205
Query executed empty at index 1434
Query executed empty at index 233
Query executed at index 1027
Query executed at index 89
Query executed empty at index 937
Query executed empty at index 1233
Query executed empty at index 271
Query executed empty at index 874
Query executed at index 82
Query executed empty at index 302
Query executed empty at index 529
Query executed at index 1084
Query executed at index 703
Query executed empty at index 445
Query executed empty at index 1332
Query executed empty at index 1520
Query executed empty at index 141
Query executed at index 161
Query executed empty at index 1030
Query executed at index 583
Query executed at index 506
Query executed at index 962
Query executed empty at index 1375
Query executed at index 1408
Query executed empty at index 794
Query is not executed at index 14
Query executed at index 725
Query executed at index 182
Query executed at index 1039
Query executed empty at in

In [23]:
sample["noised"] = False

sampled_indexes = random.sample(executed_indexes, min(100, len(executed_indexes)))


random_noisy_sample = sample_for_noise.loc[sampled_indexes]

random_noisy_sample.to_csv("../sample/dev_only_noisy.csv", index=False)

combined = pd.concat([sample, random_noisy_sample])

combined = combined.sample(frac=1).reset_index(drop=True)
combined.to_csv("../sample/dev_noisy.csv", index=False)

len(combined)


200